## Extract files

In [15]:
import os
from zipfile import ZipFile

def unzip_all_files_in_dir(data_path='data/'):
    last_files_zip = []
    while True:
        files_zip = [file for file in os.listdir(data_path) if file.endswith('.zip')]
        if files_zip == last_files_zip:
            break
        else:
            files_new = list(set(files_zip) - set(last_files_zip))
            last_files_zip = files_zip
        for file in files_new:
            ZipFile(data_path+file,'r').extractall(data_path)

## Load data

In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv', encoding="ISO-8859-1")
display(train.head(5))
display(train.describe())

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


,id,product_uid,relevance
count,74067.000000,74067.000000,74067.000000
mean,112385.709223,142331.911553,2.381634
std,64016.573650,30770.774864,0.533984
min,2.000000,100001.000000,1.000000
25%,57163.500000,115128.500000,2.000000
50%,113228.000000,137334.000000,2.330000
75%,168275.500000,166883.500000,3.000000
max,221473.000000,206650.000000,3.000000


In [2]:
attributes = pd.read_csv('data/attributes.csv')
attributes['product_uid'] = attributes['product_uid'].fillna(0).astype('int')
display(attributes.head(5))
product_descriptions = pd.read_csv('data/product_descriptions.csv')
display(product_descriptions.head(5))

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel


,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


## Exploring data

### Check null values

In [3]:
print('Total entries: ', len(train))
display(train.isnull().sum())
display(train.nunique())

Total entries:  74067


id               0
product_uid      0
product_title    0
search_term      0
relevance        0
dtype: int64

id               74067
product_uid      54667
product_title    53489
search_term      11795
relevance           13
dtype: int64

### Check if product_uid is unique

In [4]:
print(product_descriptions['product_uid'].is_unique)
print(attributes['product_uid'].is_unique)

True
False


### Before joining the attributes, I need to check why are product_uid not unique

In [5]:
attributes[attributes['product_uid'] == 100001]

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel
5,100001,Bullet06,Galvanized for extra corrosion resistance
6,100001,Bullet07,Install with 10d common nails or #9 x 1-1/2 in...
7,100001,Gauge,12
8,100001,Material,Galvanized Steel
9,100001,MFG Brand Name,Simpson Strong-Tie


In [6]:
attributes['name'].nunique()

5410

### I select only the attributes with more than 10.000 entries

In [7]:
df_showing_all_att = attributes['name'].value_counts().rename_axis('name').reset_index(name='counts')
list_showing_all_att = df_showing_all_att[df_showing_all_att['counts'] > 10000].values.tolist()
attributes_selected = [name[0] for name in list_showing_all_att]
print(attributes_selected)

for attribute in attributes_selected:
    attributes.loc[attributes['name'] == attribute, attribute] = attributes['value']

pd.set_option('display.max_columns', None)
display(attributes.head(5))

['MFG Brand Name', 'Bullet02', 'Bullet03', 'Bullet04', 'Bullet01', 'Product Width (in.)', 'Bullet05', 'Product Height (in.)', 'Product Depth (in.)', 'Product Weight (lb.)', 'Bullet06', 'Color Family', 'Bullet07', 'Material', 'Color/Finish', 'Bullet08', 'Certifications and Listings', 'Bullet09', 'Assembled Height (in.)', 'Assembled Width (in.)', 'Assembled Depth (in.)', 'Product Length (in.)', 'Bullet10', 'Indoor/Outdoor', 'Bullet11']


,product_uid,name,value,MFG Brand Name,Bullet02,Bullet03,Bullet04,Bullet01,Product Width (in.),Bullet05,Product Height (in.),Product Depth (in.),Product Weight (lb.),Bullet06,Color Family,Bullet07,Material,Color/Finish,Bullet08,Certifications and Listings,Bullet09,Assembled Height (in.),Assembled Width (in.),Assembled Depth (in.),Product Length (in.),Bullet10,Indoor/Outdoor,Bullet11
0,100001,Bullet01,Versatile connector for various 90° connection...,NaN,NaN,NaN,NaN,Versatile connector for various 90° connection...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...,NaN,Stronger than angled nailing or screw fastenin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100001,Bullet03,Help ensure joints are consistently straight a...,NaN,NaN,Help ensure joints are consistently straight a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.,NaN,NaN,NaN,Dimensions: 3 in. x 3 in. x 1-1/2 in.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100001,Bullet05,Made from 12-Gauge steel,NaN,NaN,NaN,NaN,NaN,NaN,Made from 12-Gauge steel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
attributes.drop(['name','value'], axis=1, inplace=True)

In [20]:
attributes = attributes.fillna('').groupby('product_uid').agg(lambda x: ''.join(x))
attributes.head(3)

,MFG Brand Name,Bullet02,Bullet03,Bullet04,Bullet01,Product Width (in.),Bullet05,Product Height (in.),Product Depth (in.),Product Weight (lb.),Bullet06,Color Family,Bullet07,Material,Color/Finish,Bullet08,Certifications and Listings,Bullet09,Assembled Height (in.),Assembled Width (in.),Assembled Depth (in.),Product Length (in.),Bullet10,Indoor/Outdoor,Bullet11
product_uid,,,,,,,,,,,,,,,,,,,,,,,,,
0,,,,,,,,,,,,,,,,,,,,,,,,,
100001,Simpson Strong-Tie,Stronger than angled nailing or screw fastenin...,Help ensure joints are consistently straight a...,Dimensions: 3 in. x 3 in. x 1-1/2 in.,Versatile connector for various 90° connection...,3,Made from 12-Gauge steel,3,1.5,0.26,Galvanized for extra corrosion resistance,,Install with 10d common nails or #9 x 1-1/2 in...,Galvanized Steel,,,,,,,,,,,
100002,BEHR Premium Textured DeckOver,100% acrylic solid color coating,Resists cracking and peeling and conceals spli...,"Provides a durable, mildew resistant finish","Revives wood and composite decks, railings, po...",,Covers up to 75 sq. ft. in 2 coats per gallon,,,,"Creates a textured, slip-resistant finish",Browns / Tans,"For best results, prepare with the appropriate...",,Tugboat,Actual paint colors may vary from on-screen an...,,,7.76 in,6.63 in,6.63 in,,Colors available to be tinted in most stores,,


## Join data

In [ ]:
pd.concat([train, attributes, product_descriptions], axis=1)